In [13]:
import os
from pathlib import Path

import numpy as np
import torch
from easydict import EasyDict

from src.dataset import CellDataset
from src.augmentations import train_transform
from torchvision.models.detection import maskrcnn_resnet50_fpn
from mrcnn.utils import compute_ap_range
from dotenv import load_dotenv
from src.iou_metric import iou_map
load_dotenv()

True

In [2]:
config = EasyDict(
    dataset_path=Path(os.environ['dataset_path']),
    val_size=0.2,
    device='cpu'
)


In [3]:
dataset = CellDataset(config, mode='train', transform=train_transform)
image, target = dataset[0]



In [4]:
model = maskrcnn_resnet50_fpn(num_classes=4, box_detections_per_img=500)
model.eval()

path = Path("/home/shamil/PycharmProjects/sartorius_instance_segmentation/weights/bsln70ep_flips_multiclass-2021-12-18 22_50_46.ckpt")
model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
with torch.no_grad():
    output = model.forward([image])[0]

In [26]:
def swap_columns(data):
    return data[:, [1, 0, 3, 2]]

true_masks = target['masks'].squeeze().numpy()
true_masks = np.transpose(true_masks, axes=(1,2, 0))

pred_masks = output['masks'].squeeze().numpy()
pred_masks = (pred_masks > 0.5).astype(int)
pred_masks = np.transpose(pred_masks, axes=(1, 2, 0))

pred_boxes = swap_columns(output['boxes'].numpy())

true_boxes = swap_columns(target['boxes'].numpy())

In [24]:
compute_ap_range(gt_box=true_boxes, gt_mask=true_masks, gt_class_id=target['labels'].numpy(),
                 pred_box=pred_boxes, pred_mask=pred_masks, pred_class_id=output['labels'].numpy(),
                 pred_score=output['scores'].numpy(), verbose=False)

0.20674316359380435

In [25]:
true = target['masks'].int().numpy().squeeze()
pred = output['masks'].numpy().squeeze()
pred = (pred > 0.5).astype(int)

iou_map(true, pred)

0.16499257650207869